## 1. Import Libraries

In [1]:
import numpy as np
import pandas as pd
import os, calendar 
from pathlib import Path
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import (
    Input, LSTM, Dense,
    Bidirectional, GRU, Add
)

from tensorflow.keras.optimizers import Adam
from tensorflow.keras import callbacks
from sklearn.preprocessing import MinMaxScaler, PolynomialFeatures
from sklearn.metrics import (
    mean_squared_error, mean_absolute_error, r2_score,
    accuracy_score, precision_score, recall_score, f1_score
)
import matplotlib.pyplot as plt

## 2. Parameter Initialisations

In [2]:
dataset_folder = f"datasets/UK/"
dataset = "sentinel_cams_aurn.csv"
model_dataset = "aurn_cam"
models_folder = Path(dataset_folder) / Path(model_dataset) /Path("models")
stations_file = Path(dataset_folder) / "monitoring_stations.csv"
model_weight_file = "ozone_model_lstm.weights.h5"
sequence_length=5

req_cols = [
    'Time', 'SiteNumber', 'Longitude', 'Latitude', 
    'aurn_go3_max', 'aurn_no2', 'ch4_c', 'uvbed', 'uvbedcs', 't', 'ws' # remove ch4_c 
]

engineer_cols=['aurn_go3_max', 'aurn_no2', 'ch4_c', 'uvbed', 'uvbedcs', 't', 'ws']

exclude_cols = ['Time', 'SiteNumber', 'Longitude', 'Latitude']

y_columns=['aurn_go3_max']

## 3. A Function to Get the AURN site Information

In [3]:
def get_site_info(site_number, stations_file, site_column="SiteNumber"):
    try:
        # Load the CSV
        stations_df = pd.read_csv(stations_file)
        
        # Ensure SiteNumber is treated as a string
        stations_df[site_column] = stations_df[site_column].astype(str)
        site_number = str(site_number)

        # Find the matching row
        site_info = stations_df[stations_df[site_column] == site_number]
        
        if not site_info.empty:
            site_info = site_info.iloc[0]  # Get the first match
            return site_info["SiteName"], site_info["Longitude"], site_info["Latitude"]
        else:
            return None, None, None  # Return None if not found

    except FileNotFoundError:
        print(f"Error: {stations_file} not found.")
        return None, None, None
    except Exception as e:
        print(f"Error reading site info: {e}")
        return None, None, None

## 4. Load Dataset

In [4]:
df = pd.read_csv(Path(dataset_folder)/Path(dataset), parse_dates=['Time'], dayfirst=True)

In [5]:
df['aurn_go3_max'].min(), df['aurn_go3_max'].max()

(1.0, 238.0)

In [6]:
df['aurn_go3'].min(), df['aurn_go3'].max()

(-1.0, 157.0)

#### Compute Wind Speed

In [7]:
df['ws'] = np.sqrt(df['u']**2 + df['v']**2)

#### Filter Required Site [OPTIONAL]

In [8]:
# uncomment to train a subset of the data by site
# df = df[df['SiteName'] == 'Barnsley Gawber']

#### Include Only Required Columns

In [9]:
df = df[req_cols].dropna()

In [10]:
# df['aurn_go3_max'] = np.log1p(df['aurn_go3_max'])

## 5. Feature Engineering

### Polynomial Features

In [11]:
import pandas as pd

def generate_polynomial_features(
    df, degree=2, interaction_only=False, include_bias=False, include_columns=None
):
    # Handle columns to include
    include_columns = include_columns or df.columns.tolist()  # Default to all columns if None
    included_df = df[include_columns]  # DataFrame with only included columns
    excluded_df = df.drop(columns=include_columns, errors='ignore')  # Columns not used for polynomial features

    # Generate polynomial features on the included columns
    poly = PolynomialFeatures(degree=degree, interaction_only=interaction_only, include_bias=include_bias)
    poly_features = poly.fit_transform(included_df.values)
    feature_names = poly.get_feature_names_out(included_df.columns)
    
    # Create DataFrame for polynomial features
    poly_df = pd.DataFrame(poly_features, columns=feature_names, index=df.index)
    
    # Concatenate excluded columns back with the polynomial features
    final_df = pd.concat([poly_df, excluded_df], axis=1)
    
    return final_df


df = generate_polynomial_features(
    df, 
    degree=2, 
    interaction_only=False, 
    include_bias=False, 
    include_columns=engineer_cols
)


### Temporal Features

In [12]:
def get_days_in_year(year):
    return 366 if calendar.isleap(year) else 365
        
def temporal_cyclical_features(df, time_col='Time'):
    df = df.copy()

    # Convert the time column to datetime if not already
    if not np.issubdtype(df[time_col].dtype, np.datetime64):
        df[time_col] = pd.to_datetime(df[time_col])

    # Extract temporal features
    df['Year'] = df[time_col].dt.year
    df['DayOfWeek'] = df[time_col].dt.weekday                   # Day of the week (0-6)
    df['DayOfYear'] = df[time_col].dt.dayofyear                 # Day of the year (1-365/366)
    df['Month'] = df[time_col].dt.month                         # Month of the year (1-12)
    df['IsWeekend'] = df[time_col].dt.weekday.apply(lambda x: 1 if x >= 5 else 0)  # Weekday vs Weekend
    # Extract ISO Week Number (WeekOfYear)
    df['WeekOfYear'] = df[time_col].dt.isocalendar().week
    # Handle Week 53 cases by mapping them to 52
    df['WeekOfYear'] = df['WeekOfYear'].apply(lambda x: x if x <= 52 else 52)

    # Compute the Lunar Phase (0 = New Moon, 14-15 = Full Moon, 29 = Next New Moon)
    df['LunarDay'] = (df[time_col] - pd.Timestamp("2000-01-06")).dt.days % 29.53  # Reference New Moon date

    # Determine the season (0: Winter, 1: Spring, 2: Summer, 3: Fall)
    df['Season'] = df[time_col].apply(lambda x: 
        0 if x.month in [12, 1, 2] else 
        1 if x.month in [3, 4, 5] else 
        2 if x.month in [6, 7, 8] else 
        3
    )

    df['DaysInYear'] = df['Year'].apply(get_days_in_year)

    # Cyclical encoding for the day of the year
    df['DayOfYearSin'] = np.sin(2 * np.pi * df['DayOfYear'] / df['DaysInYear'])
    df['DayOfYearCos'] = np.cos(2 * np.pi * df['DayOfYear'] / df['DaysInYear'])

    df['DayOfWeekSin'] = np.sin(2 * np.pi * df['DayOfWeek'] / 7)
    df['DayOfWeekCos'] = np.cos(2 * np.pi * df['DayOfWeek'] / 7)

    df['MonthSin'] = np.sin(2 * np.pi * df['Month'] / 12)
    df['MonthCos'] = np.cos(2 * np.pi * df['Month'] / 12)

    df['WeekOfYearSin'] = np.sin(2 * np.pi * df['WeekOfYear'] / 52)
    df['WeekOfYearCos'] = np.cos(2 * np.pi * df['WeekOfYear'] / 52)

    # Cyclical Encoding of the Lunar Cycle
    df['LunarSin'] = np.sin(2 * np.pi * df['LunarDay'] / 29.53)
    df['LunarCos'] = np.cos(2 * np.pi * df['LunarDay'] / 29.53)

    
    return df


df = temporal_cyclical_features(df, time_col='Time')

### Min/Max Normalisation

In [13]:
def min_max_scale_features(df, y_columns, exclude_x_columns=None):
    # Separate features and target
    if exclude_x_columns:
        y_columns = y_columns + exclude_x_columns
        
    features = df.drop(columns=y_columns)
    target = df[y_columns]

    # Apply Min-Max Scaling to features
    scaler = MinMaxScaler()
    scaled_features = scaler.fit_transform(features)
    
    # Convert back to DataFrame and retain original column names
    scaled_features_df = pd.DataFrame(scaled_features, columns=features.columns, index=df.index)
    
    # Combine scaled features with the target column
    result_df = pd.concat([scaled_features_df, target], axis=1)
    
    return result_df


df = min_max_scale_features(df, y_columns=y_columns, exclude_x_columns=exclude_cols)

### Split the Dataset

In [14]:
train_start = '2019-01-01'
train_end = '2023-12-31'

# The training set split (includes train and validation)
df_train = df[(df['Time'] >= train_start) & (df['Time'] <= train_end)]

In [15]:
test_start = '2024-01-01'
test_end = '2024-08-31'

# The training set split (includes train and validation)
df_test = df[(df['Time'] >= test_start) & (df['Time'] <= test_end)]

### Create Training Sequence

In [16]:
from tqdm import tqdm
def create_sequences(df, seq_length, y_columns):
    sequences = []
    targets = []
    metadata = []  # List to store Time and SiteNumber information
    
    for i in range(len(df) - seq_length):
        seq = df.iloc[i:i + seq_length]
        # Check if the sequence has consecutive days
        time_diff = (seq['Time'].diff().dropna() == pd.Timedelta(days=1)).all()
        
        if not time_diff:  # Skip sequences with non-consecutive days
            continue

        # Include lagged y_column(s) as features within the sequence
        lagged_features = seq[y_columns].values  # Extract the lagged `y_columns`
        other_features = seq[df.columns.drop(['Time', 'SiteNumber', *y_columns])].values
        sequence_features = np.hstack((other_features, lagged_features))  # Concatenate features

        
        # Append the valid sequence and target
        sequences.append(seq[df.columns.drop(['Time', 'SiteNumber'])].values)
        targets.append(df.iloc[i + seq_length][y_columns])
        
        # Store metadata for sequence
        metadata.append({
            "Start_Time": seq.iloc[0]['Time'],  # Start time of the sequence
            "End_Time": seq.iloc[-1]['Time'],  # End time of the sequence
            "SiteNumber": seq.iloc[0]['SiteNumber'],  # Site number for the sequence
        })
    
    metadata_df = pd.DataFrame(metadata)
    
    return np.array(sequences), np.array(targets), metadata_df



def generate_sequences_by_site(df, site_column, seq_length, y_columns):
    all_sequences = []
    all_targets = []
    all_metadatas = []

    # Iterate over each unique site
    for site, group in tqdm(df.groupby(site_column), desc="Processing Sites"):
        # Sort the group by time
        group = group.sort_values(by='Time')

        # Create sequences for this group
        site_sequences, site_targets, site_metadata = create_sequences(group, seq_length, y_columns)

        # Append sequences and targets
        if site_sequences.size > 0:  # Only add non-empty sequences
            all_sequences.append(site_sequences)
            all_targets.append(site_targets)
            all_metadatas.append(site_metadata)

    # Concatenate sequences and targets across all sites
    if all_sequences:
        all_sequences = np.concatenate(all_sequences, axis=0)
        all_targets = np.concatenate(all_targets, axis=0)
        all_metadatas = np.concatenate(all_metadatas, axis=0)
    else:
        all_sequences = np.array([])
        all_targets = np.array([])
        all_metadatas = np.array([])

    # Return sequences and targets
    return all_sequences, all_targets, all_metadatas


X_train, y_train, X_train_meta = generate_sequences_by_site(df_train, 'SiteNumber', sequence_length, y_columns)
X_test, y_test, X_test_meta = generate_sequences_by_site(df_test, 'SiteNumber', sequence_length, y_columns)


Processing Sites: 100%|████████████████████████████████████████████████████████████████| 78/78 [00:17<00:00,  4.39it/s]


In [17]:
# Ensure data types are float32
X_train = X_train.astype('float32')
y_train = y_train.astype('float32')
X_test = X_test.astype('float32')
y_test = y_test.astype('float32')

In [18]:
X_train.shape, y_train.shape, X_train_meta.shape,  X_test.shape, y_test.shape, X_test_meta.shape

((100244, 5, 56),
 (100244, 1),
 (100244, 3),
 (12352, 5, 56),
 (12352, 1),
 (12352, 3))

## 6. ML Models 

### 1. LSTM 

In [54]:
# LSTM-based model
def lstm_model(in_time_steps, in_features):
    input_layer = Input(shape=(in_time_steps, in_features))

    # First bi-directional LSTM layer
    x =  LSTM(units=50, return_sequences=True)(input_layer)
    x =  LSTM(units=50, return_sequences=False)(x)
    
    x = Dense(units=10, activation='relu')(x)
    output_layer = Dense(1, activation='linear')(x)
    
    # Create the model
    model = Model(inputs=input_layer, outputs=output_layer)

    return model

### 2. Bi-Directional LSTM

In [55]:
# bi-directional LSTM-based model
def bi_lstm_model(in_time_steps, in_features):
    input_layer = Input(shape=(in_time_steps, in_features))

    # First bi-directional LSTM layer
    x =  Bidirectional(LSTM(units=256, return_sequences=True))(input_layer)
    
    # Feed-forward layer on the sequence
    x_ffn = Dense(units=512, activation='relu')(x)  # Apply Dense layer to sequence
    x =  Bidirectional(LSTM(units=256, return_sequences=True))(x)
    
    # Residual connection: add bi-directional LSTM output and Dense output
    x = Add()([x, x_ffn])
    
    # Final bi-directional LSTM layers for sequence compression
    x =  Bidirectional(LSTM(units=128, return_sequences=True))(x)
    x =  Bidirectional(LSTM(units=64, return_sequences=True))(x)
    x =  Bidirectional(LSTM(units=32, return_sequences=False))(x)
    
    x = Dense(units=10, activation='relu')(x)
    
    # Output layer (regression target)
    output_layer = Dense(1, activation='linear')(x)
    
    # Create the model
    model = Model(inputs=input_layer, outputs=output_layer)

    return model

### 3. Gated Recurrent Unit (GRU)

In [56]:
# Gated Recurrent Unit (GRU)
def gru_model(in_time_steps, in_features):
    input_layer = Input(shape=(in_time_steps, in_features))

    # First bi-directional LSTM layer
    x =  GRU(units=50, return_sequences=True)(input_layer)
    x =  GRU(units=50, return_sequences=False)(x)
    
    x = Dense(units=10, activation='relu')(x)
    
    # Output layer (regression target)
    output_layer = Dense(1, activation='linear')(x)
    
    # Create the model
    model = Model(inputs=input_layer, outputs=output_layer)

    return model

### 4. Bi-Directional Gated Recurrent Unit (GRU)

In [57]:
# bi-directional GRU-based model
def bi_gru_model(in_time_steps, in_features):
    input_layer = Input(shape=(in_time_steps, in_features))

    # First bi-directional LSTM layer
    x =  Bidirectional(GRU(units=256, return_sequences=True))(input_layer)
    
    # Feed-forward layer on the sequence
    x_ffn = Dense(units=512, activation='relu')(x)  # Apply Dense layer to sequence
    x =  Bidirectional(GRU(units=256, return_sequences=True))(x)
    
    # Residual connection: add bi-directional LSTM output and Dense output
    x = Add()([x, x_ffn])
    
    # Final bi-directional LSTM layers for sequence compression
    x =  Bidirectional(GRU(units=128, return_sequences=True))(x)
    x =  Bidirectional(GRU(units=64, return_sequences=True))(x)
    x =  Bidirectional(GRU(units=32, return_sequences=False))(x)
    
    x = Dense(units=10, activation='relu')(x)
    
    # Output layer (regression target)
    output_layer = Dense(1, activation='linear')(x)
    
    # Create the model
    model = Model(inputs=input_layer, outputs=output_layer)

    return model

### 4. Train and Save Models

In [25]:
def train_and_save_models(models_folder, sequence_length, X_train, y_train, X_test, y_test, epochs=500):
    models = {
        "lstm": lstm_model(sequence_length, X_train.shape[2]),
        "bi_lstm": bi_lstm_model(sequence_length, X_train.shape[2]),
        "gru": gru_model(sequence_length, X_train.shape[2]),
        "bi_gru": bi_gru_model(sequence_length, X_train.shape[2]),
    }
    
    os.makedirs(Path(models_folder), exist_ok=True)
    huber = tf.keras.losses.Huber(delta=0.1)
    
    for model_name, model in models.items():
        print(f"Training {model_name} model...")
        model.compile(optimizer="adam", loss=huber)
        
        callbacks_list = [
            callbacks.ModelCheckpoint(
                filepath=Path(models_folder) / f"{model_name}.h5",
                monitor="val_loss",
                verbose=1,
                save_weights_only=False,
                save_best_only=True,
            ),
            callbacks.ReduceLROnPlateau(
                monitor='val_loss',
                factor=0.1,
                patience=10,
                min_lr=0.0
            ),
            callbacks.EarlyStopping(
                monitor='val_loss',        
                patience=15,                
                min_delta=0.001,           
                restore_best_weights=True   
            )            
        ]
        
        model.fit(
            X_train, y_train,
            validation_data=(X_test, y_test),
            epochs=epochs,
            batch_size=32, ## TODO: Change
            callbacks=callbacks_list
        )
        
        model.save(Path(models_folder) / f"{model_name}.h5")
        print(f"{model_name} model saved!")

# train the model
train_and_save_models(
    models_folder, 
    sequence_length, 
    X_train, y_train, 
    X_test, y_test,
    epochs=200
)

Training lstm model...
Epoch 1/200
3129/3133 [============================>.] - ETA: 0s - loss: 1.3029     
Epoch 1: val_loss improved from inf to 0.92418, saving model to datasets\UK\aurn_cam\models\lstm.h5
3133/3133 [==============================] - 34s 9ms/step - loss: 1.3025 - val_loss: 0.9242 - lr: 0.0010
Epoch 2/200
3127/3133 [============================>.] - ETA: 0s - loss: 0.9523 
Epoch 2: val_loss improved from 0.92418 to 0.90189, saving model to datasets\UK\aurn_cam\models\lstm.h5
3133/3133 [==============================] - 27s 9ms/step - loss: 0.9522 - val_loss: 0.9019 - lr: 0.0010
Epoch 3/200
3130/3133 [============================>.] - ETA: 0s - loss: 0.9366 
Epoch 3: val_loss improved from 0.90189 to 0.86900, saving model to datasets\UK\aurn_cam\models\lstm.h5
3133/3133 [==============================] - 26s 8ms/step - loss: 0.9366 - val_loss: 0.8690 - lr: 0.0010
Epoch 4/200
3130/3133 [============================>.] - ETA: 0s - loss: 0.9288 
Epoch 4: val_loss did not 

### 5. Evaluate Models

In [24]:
def evaluate_all_models(models_folder, X_test, y_test, X_test_meta, output_dir, site_column="SiteNumber"):
       
    models = {model_name: tf.keras.models.load_model(Path(models_folder) / f"{model_name}.h5") 
              for model_name in ["lstm", "bi_lstm", "gru", "bi_gru"]}
    
    for model_name, model in models.items():
        output_folder = output_dir/Path(model_name)
        Path(output_folder).mkdir(parents=True, exist_ok=True)
        
        print(f"Evaluating {model_name} model...")
        results = []
        predictions_list = []  # Store predictions for CSV
    
        y_pred = model.predict(X_test)
        
        metrics = {
            "Model": model_name,
            "RMSE": np.sqrt(mean_squared_error(y_test, y_pred)),
            "MAE": mean_absolute_error(y_test, y_pred),
            "MAPE": np.mean(np.abs((y_test - y_pred) / y_test)) * 100,
            "R-squared": r2_score(y_test, y_pred)
        }
        results.append(metrics)
        pd.DataFrame(results).to_csv(Path(output_folder) / f"{model_name}_metrics.csv", index=False)
        

        if isinstance(X_test_meta, np.ndarray):
            expected_columns=["Start_Time", "End_Time", "SiteNumber"]
            X_test_meta = pd.DataFrame(X_test_meta, columns=expected_columns)
        
        # Collect data for CSV
        predictions_df = pd.DataFrame({
            "Time": X_test_meta["End_Time"].values.flatten(),
            "SiteNumber": X_test_meta["SiteNumber"].values.flatten(),
            "True_Value": y_test.flatten(),
            "Predicted_Value": y_pred.flatten()
        })
        predictions_list.append(predictions_df)
        
        plt.figure(figsize=(10, 6))
        plt.plot(X_test_meta["End_Time"], y_test, label="True Values", color="blue")
        plt.plot(X_test_meta["End_Time"], y_pred.flatten(), label="Predictions", color="red")
        plt.title(f"{model_name} Predictions vs True Values")
        plt.xlabel("Dates")
        plt.ylabel("Ground Ozone")
        plt.legend()
        plt.grid()
        plt.savefig(Path(output_folder) / f"{model_name}_predictions.pdf", format="pdf")
        plt.close()
        
        
        # Save all predictions for this model
        all_predictions_df = pd.concat(predictions_list, ignore_index=True)
        
        # Apply function to each row to get the SiteName, Longitude and Latitudes
        all_predictions_df[['SiteName', 'Longitude', 'Latitude']] = all_predictions_df['SiteNumber'].apply(
            lambda x: pd.Series(get_site_info(x, stations_file))
        )

        all_predictions_df.to_csv(output_folder / f"{model_name}_predictions.csv", index=False)

    print(f"Evaluation results saved in directory: {output_folder}")


output_dir=Path(dataset_folder)/Path(model_dataset)/Path('results')
evaluate_all_models(
    models_folder, 
    X_test, y_test, X_test_meta, 
    output_dir=output_dir,
    site_column="SiteNumber"
)

Evaluating lstm model...
386/386 [==============================] - 2s 4ms/step
Evaluating bi_lstm model...
386/386 [==============================] - 6s 10ms/step
Evaluating gru model...
386/386 [==============================] - 1s 3ms/step
Evaluating bi_gru model...
386/386 [==============================] - 4s 5ms/step
Evaluation results saved in directory: datasets\UK\aurn_cam\results\bi_gru


### 6. Ensemble

In [58]:
from keras.layers import Average
from keras.models import Model

def ensemble_model(models):
    # Get the inputs and outputs of the individual models
    inputs = [model.input for model in models]
    predictions = [model.output for model in models]
    
    # Average the predictions
    ensemble_output = Average()(predictions)
    
    # Create the ensemble model
    ensemble = Model(inputs=inputs, outputs=ensemble_output)
    
    return ensemble


In [ ]:
def train_ensemble_model(X_train, y_train, X_test, y_test, sequence_length, models_folder, model_fns, epochs=500):
    # Create each model dynamically using the passed functions
    models = [model_fn(sequence_length, X_train.shape[2]) for model_fn in model_fns]

    # Create the ensemble model
    model = ensemble_model(models)  # Assuming ensemble_model function takes a list of models

    # Define the loss function and compile the model
    huber = tf.keras.losses.Huber(delta=0.1)
    model.compile(optimizer='adam', loss=huber)

    # Create the model directory if it doesn't exist
    os.makedirs(Path(models_folder), exist_ok=True)
    path_checkpoint = Path(models_folder) / Path("ensemble.h5")

    # Define the callbacks
    reduce_lr = callbacks.ReduceLROnPlateau(
        monitor='val_loss', 
        factor=0.1, 
        patience=10, 
        min_lr=0.0
    )

    modelckpt_callback = callbacks.ModelCheckpoint(
        monitor="val_loss",
        filepath=path_checkpoint,
        verbose=1,
        save_weights_only=False,
        save_best_only=True,
    )

    early_stopping = callbacks.EarlyStopping(
        monitor='val_loss',        
        patience=15,                
        min_delta=0.001,           
        restore_best_weights=True   
    )

    # Train the model
    history = model.fit(
        [X_train, X_train, X_train, X_train], y_train, 
        validation_data=([X_test, X_test, X_test, X_test], y_test), 
        epochs=epochs, 
        batch_size=32, 
        callbacks=[modelckpt_callback, reduce_lr, early_stopping]
    )
    
    return history



model_fns = [lstm_model, bi_lstm_model, gru_model, bi_gru_model]  # Pass model creation functions as a list
history = train_ensemble_model(
    X_train, y_train, 
    X_test, y_test, 
    sequence_length, 
    models_folder, 
    model_fns, 
    epochs=500
)


Epoch 1/500
3133/3133 [==============================] - ETA: 0s - loss: 1.2828      
Epoch 1: val_loss improved from inf to 0.91004, saving model to datasets\UK\aurn_cam\models\ensemble.h5
3133/3133 [==============================] - 179s 50ms/step - loss: 1.2828 - val_loss: 0.9100 - lr: 0.0010
Epoch 2/500
2862/3133 [==========================>...] - ETA: 12s - loss: 0.9543 

### 7. Evaluate Ensemble

In [25]:
def evaluate_ensemble(models_folder, X_test, y_test, X_test_meta, output_dir, model_name, site_column="SiteNumber"):
    print(Path(models_folder) / f"{model_name}.h5")
    model = tf.keras.models.load_model(Path(models_folder) / f"{model_name}.h5") 
    
    output_folder = output_dir/Path(model_name)
    Path(output_folder).mkdir(parents=True, exist_ok=True)
        
    print(f"Evaluating {model_name} model...")
    results = []
    predictions_list = []
        
    y_pred = model.predict([X_test, X_test, X_test, X_test])
    
    metrics = {
        "Model": model_name,
        "RMSE": np.sqrt(mean_squared_error(y_test, y_pred)),
        "MAE": mean_absolute_error(y_test, y_pred),
        "MAPE": np.mean(np.abs((y_test - y_pred) / y_test)) * 100,
        "R-squared": r2_score(y_test, y_pred)
    }
    results.append(metrics)
    pd.DataFrame(results).to_csv(Path(output_folder) / f"{model_name}_metrics.csv", index=False)

    if isinstance(X_test_meta, np.ndarray):
            expected_columns=["Start_Time", "End_Time", "SiteNumber"]
            X_test_meta = pd.DataFrame(X_test_meta, columns=expected_columns)
    
    # Collect data for CSV
    predictions_df = pd.DataFrame({
        "Time": X_test_meta["End_Time"].values.flatten(),
        "SiteNumber": X_test_meta["SiteNumber"].values.flatten(),
        "True_Value": y_test.flatten(),
        "Predicted_Value": y_pred.flatten()
    })
    predictions_list.append(predictions_df)
        
    # Save all predictions for this model
    all_predictions_df = pd.concat(predictions_list, ignore_index=True)
    
    # Apply function to each row to get the SiteName, Longitude and Latitudes
    all_predictions_df[['SiteName', 'Longitude', 'Latitude']] = all_predictions_df['SiteNumber'].apply(
        lambda x: pd.Series(get_site_info(x, stations_file))
    )

    all_predictions_df.to_csv(output_folder / f"{model_name}_predictions.csv", index=False)

    print(f"Evaluation results saved in directory: {output_folder}")


output_dir=Path(dataset_folder)/Path(model_dataset)/Path('results')
evaluate_ensemble(
    models_folder, 
    X_test, y_test, X_test_meta,
    output_dir, 
    model_name="ensemble",
    site_column="SiteNumber"
)

datasets\UK\aurn_cam\models\ensemble.h5
Evaluating ensemble model...
386/386 [==============================] - 12s 20ms/step
Evaluation results saved in directory: datasets\UK\aurn_cam\results\ensemble


## 7. Final: Post Processing of Results
The models we have develop predict as single value for Ozone. Can the model performance be improved by binning the prediction into deciles similar to the one used by AURN for air pollution.

In [26]:
def get_bin_numbers(go3, go3_min=1, go3_max=238, bins=4):
    # Normalize the value of go3 in the range [0, 1]
    normalized_value = (go3 - go3_min) / (go3_max - go3_min)
    
    # Multiply by the number of bins to get a value between 0 and bins-1
    decile = normalized_value * bins
    
    # Return the decile number (rounded to the nearest integer)
    return min(bins - 1, round(decile))  # Ensure the decile is within the range [0, bins-1]


In [27]:
for model_name in ["lstm", "bi_lstm", "gru", "bi_gru", "ensemble"]:
    # Define the path for the output directory and the predictions CSV file
    output_dir = Path(dataset_folder)/Path(model_dataset)/Path('results') / model_name / f"{model_name}_predictions.csv"
    decile_metrics_file = Path(dataset_folder)/Path(model_dataset)/Path('results') / model_name / f"{model_name}__metrics_deciles.csv"
    
    # Open the CSV as a DataFrame
    df = pd.read_csv(output_dir)
    
    # Apply the deciles calculation using the 'go3' column
    df['Predicted_Value_Decile'] = df['Predicted_Value'].apply(get_bin_numbers)
    df['True_Value_Decile'] = df['True_Value'].apply(get_bin_numbers)

    # Save the DataFrame back to the same file
    df.to_csv(output_dir, index=False)
    
    # print or log a message indicating that the deciles were applied and the file was saved
    print(f"Deciles added for {model_name} and saved to {output_dir}")

    # Ensure necessary columns exist
    if 'True_Value' in df.columns and 'Predicted_Value' in df.columns:
        y_true = df['True_Value_Decile']
        y_pred = df['Predicted_Value_Decile']
        
       # Compute classification metrics
        metrics = {
            "Model": model_name,
            "Accuracy": accuracy_score(y_true, y_pred),
            "Precision": precision_score(y_true, y_pred, average='weighted', zero_division=0),
            "Recall": recall_score(y_true, y_pred, average='weighted', zero_division=0),
            "F1-Score": f1_score(y_true, y_pred, average='weighted', zero_division=0)
        }

        # Save metrics to a CSV file
        metrics_df = pd.DataFrame([metrics])
        metrics_df.to_csv(decile_metrics_file, header=True, index=False)


Deciles added for lstm and saved to datasets\UK\aurn_cam\results\lstm\lstm_predictions.csv
Deciles added for bi_lstm and saved to datasets\UK\aurn_cam\results\bi_lstm\bi_lstm_predictions.csv
Deciles added for gru and saved to datasets\UK\aurn_cam\results\gru\gru_predictions.csv
Deciles added for bi_gru and saved to datasets\UK\aurn_cam\results\bi_gru\bi_gru_predictions.csv
Deciles added for ensemble and saved to datasets\UK\aurn_cam\results\ensemble\ensemble_predictions.csv
